In [2]:
'''
This is a documenation of a possible implementation of pachyderm into Jupyter
and see whether we can implement data versioning to a certain extent
let us list the various tasks of the mini project and implement it along the way:
Before going into any details, make sure you have the necessary stuff installed
for this.
1. We create a repo for the initial raw data. We will call it SMSTRAINING. This
   is where we will be putting different versions of input data (since this particular
   data set only has two columns, we will be giving more focus to the amount of data,
   i.e. varying the the number of rows)
'''


'\nThis is a documenation of a possible implementation of pachyderm into Jupyter\nand see whether we can implement data versioning to a certain extent\nlet us list the various tasks of the mini project and implement it along the way:\nBefore going into any details, make sure you have the necessary stuff installed\nfor this.\n1. We create a repo for the initial raw data. We will call it SMSTRAINING. This\n   is where we will be putting different versions of input data (since this particular\n   data set only has two columns, we will be giving more focus to the amount of data,\n   i.e. varying the the number of rows)\n'

In [30]:
#We create a new Repo through terminal commands

!pachctl create repo SMSTRAINING
!pachctl list repo

NAME        CREATED                SIZE (MASTER) 
SMSTRAINING Less than a second ago 0B            
jup         2 hours ago            0B            
test        23 hours ago           0B            
prep2       25 hours ago           868.1KiB      
prep        27 hours ago           555KiB        
sms         27 hours ago           473.5KiB      
possible    43 hours ago           55B           
inference   43 hours ago           94B           
kim         44 hours ago           1.465KiB      
training    44 hours ago           3.882KiB      


In [31]:
#Now we read through the data and add different versions of it to the data set
import pandas as pd
data = pd.read_csv("EK0bZ7sBRRiKU2kKs6fx_spamdata-1562916291191.csv")
print(len(data))


5572


In [32]:
#there are 5572 rows in the data set, so we will put in three versions containg the following sizes:
# 4000, 5000, 5572
data1 = data[:-1572]
data2 = data[:-572]
data1.to_csv("raw.csv")
!pachctl put file SMSTRAINING@master:raw.csv -f raw.csv --overwrite


In [33]:
#we put in the rest of the files
data2.to_csv("raw.csv")
!pachctl put file SMSTRAINING@master:raw.csv -f raw.csv --overwrite


In [34]:
data.to_csv("raw.csv")
!pachctl put file SMSTRAINING@master:raw.csv -f raw.csv --overwrite


In [35]:
data3 = data[:-2572]
data3.to_csv("raw.csv")
!pachctl put file SMSTRAINING@master:raw.csv -f raw.csv --overwrite


In [36]:
#We now have four versions of the data, lets have a look at them
!pachctl list commit SMSTRAINING

REPO        BRANCH COMMIT                           PARENT                           STARTED            DURATION           SIZE     
SMSTRAINING master 1c81adf333904bb8afc7175474f27a21 7f18d48fa86a401896ed2adb0f1fa118 24 seconds ago     Less than a second 268.4KiB 
SMSTRAINING master 7f18d48fa86a401896ed2adb0f1fa118 534750817c0846639aeb930432a45d90 58 seconds ago     Less than a second 494.2KiB 
SMSTRAINING master 534750817c0846639aeb930432a45d90 d5627bffdfb34ea0acbbc53388e4d6b0 About a minute ago Less than a second 444.5KiB 
SMSTRAINING master d5627bffdfb34ea0acbbc53388e4d6b0 <none>                           2 minutes ago      Less than a second 354.6KiB 


In [37]:
'''
You can look at in the way as if we were given four versions of the raw data where some of them are bad and 
we can revert back to the older version using the data versionality service of pachyderm. I have purposefully made 
the current most data the worst one to demonstrate it
'''

'\nYou can look at in the way as if we were given four versions of the raw data where some of them are bad and \nwe can revert back to the older version using the data versionality service of pachyderm. I have purposefully made \nthe current most data the worst one to demonstrate it\n'

In [38]:
'''
2. It is now time to preprocess the data, i.e., clean the data and/or apply some feature engineering to it
   Like the last version, we will have two versions, one where there are less features than the other one
   just to mess around with data a little more.
'''

'\n2. It is now time to preprocess the data, i.e., clean the data and/or apply some feature engineering to it\n   Like the last version, we will have two versions, one where there are less features than the other one\n   just to mess around with data a little more.\n'

In [39]:
#version one: a more enhancend cleaning and feature engineering
#it is time to clean the text data
import string
punctuations = string.punctuation

from nltk.corpus import stopwords
stopword = stopwords.words("english")

from nltk.stem.wordnet import WordNetLemmatizer
lem = WordNetLemmatizer()


def cleanText(text):
    #first convert to lower case
    cleanText = text.lower()
    #removing the punctuations
    cleanText = "".join(word for word in cleanText if word not in punctuations)
    #time to remove the stopwords as they are basically noise
    words = cleanText.split()
    words = [word for word in words if word not in stopword]
    cleanText = " ".join(words)
    #time to perform lemmatization
    words = [lem.lemmatize(word, "v") for word in words]
    words = [lem.lemmatize(word, "n") for word in words]
    return cleanText

#testing the function out
cleanText("I will, be playing a game today!")

'playing game today'

In [21]:
#now lets have a look at the current raw data we have
!pachctl get file SMSTRAINING@master:raw.csv

,label,text
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"
5,spam,"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv"
6,ham,Even my brother is not like to speak with me. They treat me like aids patent.
7,ham,As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
8,spam,WINNER!! As a valued network customer you have been selected to receivea å£900 prize reward! To claim call 09061701461. Claim c

In [40]:
#There are some ways to get the data out of pachyderm, but they are kind of confusing. We will treat it as an output
#the terminal screen and we will save the output in a csv file which we can access easily
!pachctl get file SMSTRAINING@master:raw.csv > m.csv


In [55]:
#Now read the data normally through the command .read_csv()
rawData = pd.read_csv("m.csv", index_col = False)
rawData = rawData.drop("Unnamed: 0", axis = 1)


In [56]:
#Now that we have a version of the raw data, we go about pre-processing it and performing some feature engineering
import nltk
pos_dic = {"noun": ["NNP" , "NN", "NNS", "NNPS"], "verb" : ["VBZ", "VB", "VBD", "VBN", "VBG"]}

def partOfSpeechTag(text, family):
    #tokenize the sentence
    tags = nltk.pos_tag(nltk.word_tokenize(text))
    count  = 0
    #get the tags of the tokenized words
    for tag in tags:
        #get the part of speech tag
        tag = tag[1]
        #check if it is present in the predefined dictionary of ours
        if tag in pos_dic[family]:
            count += 1
    return count

#test the function out
partOfSpeechTag("They are playing in the ground", "verb")

1

In [57]:
def preProcessAndFeature(rawData):
    rawData["cleanText"] = rawData["text"].apply(cleanText)
    rawData["rawWordCount"] = rawData["text"].apply(lambda x : len(x.split()))
    rawData["cleanWordCount"] = rawData["cleanText"].apply(lambda x : len(x.split()))
    rawData['characterCount'] = rawData['text'].apply(lambda x :len(x))
    rawData['characterCountWithoutSpace'] = rawData['text'].apply(lambda x :len(x.replace(" ", "")))
    rawData["digitOccurrece"] = rawData["text"].apply(lambda x: sum([1 if word.isdigit() else 0 for word in x.split()]))
    rawData["nounCount"] = rawData["text"].apply(lambda x : partOfSpeechTag(x, "noun"))
    rawData["verbCount"] = rawData["text"].apply(lambda x : partOfSpeechTag(x, "verb"))
    return rawData
rawData = preProcessAndFeature(rawData)


In [58]:
#Now this is the preprocesssed data
rawData

,label,text,cleanText,rawWordCount,cleanWordCount,characterCount,characterCountWithoutSpace,digitOccurrece,nounCount,verbCount
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...,20,16,111,92,0,10,1
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni,6,6,29,24,0,4,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...,28,23,155,128,2,13,3
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say,11,9,49,39,0,3,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah dont think goes usf lives around though,13,8,61,49,0,1,4
5,spam,FreeMsg Hey there darling it's been 3 week's n...,freemsg hey darling 3 weeks word back id like ...,32,19,148,117,1,9,8
6,ham,Even my brother is not like to speak with me. ...,even brother like speak treat like aids patent,16,8,77,62,0,3,2
7,ham,As per your request 'Melle Melle (Oru Minnamin...,per request melle melle oru minnaminunginte nu...,26,16,160,135,0,12,4
8,spam,WINNER!! As a valued network customer you have...,winner valued network customer selected receiv...,26,18,158,133,1,11,6
9,spam,Had your mobile 11 months or more? U R entitle...,mobile 11 months u r entitled update latest co...,29,18,154,126,2,11,3


In [59]:
#It is now time to store the more versioned data in another repo called CLEANEDDATA
!pachctl create repo CLEANDATA
rawData.to_csv("processedData.csv")
!pachctl put file CLEANDATA@master:processedData.csv -f processedData.csv --overwrite

In [60]:
#Let us just make sure that the commit was successful
!pachctl list repo

NAME        CREATED        SIZE (MASTER) 
CLEANDATA   36 seconds ago 483.7KiB      
SMSTRAINING 23 minutes ago 268.4KiB      
jup         2 hours ago    0B            
test        24 hours ago   0B            
prep2       26 hours ago   868.1KiB      
prep        28 hours ago   555KiB        
sms         28 hours ago   473.5KiB      
possible    44 hours ago   55B           
inference   44 hours ago   94B           
kim         44 hours ago   1.465KiB      
training    45 hours ago   3.882KiB      


In [61]:
#As we can see, the file was successfully committed into the repo
#Now as our second step of the overall second step, we put another version of the text into the repo which is not that
#much preprocessed
def preProcessAndFeatureVersion2(rawData):
    rawData["cleanText"] = rawData["text"].apply(cleanText)
    rawData["cleanWordCount"] = rawData["cleanText"].apply(lambda x : len(x.split()))
    rawData["nounCount"] = rawData["text"].apply(lambda x : partOfSpeechTag(x, "noun"))
    rawData["verbCount"] = rawData["text"].apply(lambda x : partOfSpeechTag(x, "verb"))
    return rawData

In [63]:
secondVersionRawData = pd.read_csv("m.csv", index_col = False)
secondVersionRawData = secondVersionRawData.drop("Unnamed: 0", axis = 1)
secondVersionRawData = preProcessAndFeatureVersion2(secondVersionRawData)
#check whether it came out as expected
secondVersionRawData

,label,text,cleanText,cleanWordCount,nounCount,verbCount
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...,16,10,1
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni,6,4,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...,23,13,3
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say,9,3,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah dont think goes usf lives around though,8,1,4
5,spam,FreeMsg Hey there darling it's been 3 week's n...,freemsg hey darling 3 weeks word back id like ...,19,9,8
6,ham,Even my brother is not like to speak with me. ...,even brother like speak treat like aids patent,8,3,2
7,ham,As per your request 'Melle Melle (Oru Minnamin...,per request melle melle oru minnaminunginte nu...,16,12,4
8,spam,WINNER!! As a valued network customer you have...,winner valued network customer selected receiv...,18,11,6
9,spam,Had your mobile 11 months or more? U R entitle...,mobile 11 months u r entitled update latest co...,18,11,3


In [64]:
#Yes the second version has also come out correctly, now it is time to commit this as well
secondVersionRawData.to_csv("processedData.csv")
!pachctl put file CLEANDATA@master:processedData.csv -f processedData.csv --overwrite

In [65]:
#Let us just make sure that the commit was successful
!pachctl list repo

NAME        CREATED        SIZE (MASTER) 
CLEANDATA   6 minutes ago  450.9KiB      
SMSTRAINING 29 minutes ago 268.4KiB      
jup         2 hours ago    0B            
test        24 hours ago   0B            
prep2       26 hours ago   868.1KiB      
prep        28 hours ago   555KiB        
sms         28 hours ago   473.5KiB      
possible    44 hours ago   55B           
inference   44 hours ago   94B           
kim         45 hours ago   1.465KiB      
training    45 hours ago   3.882KiB      


In [66]:
#as you can see, it was successful as the size of the repo CLEANDATA changed from roughly 483 KiB to 450 KiB

In [67]:
'''
3. Now that we have access to the preprocessed data it is time to analyze it a bit more by some extensive Feature 
   Engineering methods such as TfIDF and so on.
'''

'\n3. Now that we have access to the preprocessed data it is time to analyze it a bit more by some extensive Feature \n   Engineering methods such as TfIDF and so on.\n'

In [68]:
#first let us have a look at the commits that have been made to the CLEANDATA
!pachctl list commit CLEANDATA

REPO      BRANCH COMMIT                           PARENT                           STARTED        DURATION           SIZE     
CLEANDATA master a6f41f8ba4a54824a5a9e4bf948a9bb5 b290f1d7f77c487cb4bcc066ba731ab5 7 minutes ago  Less than a second 450.9KiB 
CLEANDATA master b290f1d7f77c487cb4bcc066ba731ab5 <none>                           13 minutes ago Less than a second 483.7KiB 


In [69]:
#there are two versions of the preprocessed data: the idea here was to showcase an event where some changes have been 
#made to a certain preprocessing model like inclusion of certain features and how we can refer back to the previous
#data so easly through Pachyderm

#our first step is to access the file from the CLEANDATA REPO
!pachctl get file CLEANDATA@master:processedData.csv > pre.csv

In [72]:
#Now read the data normally through the command .read_csv()
processedData = pd.read_csv("pre.csv", index_col = False)
processedData = processedData.drop("Unnamed: 0", axis = 1)
processedData

,label,text,cleanText,cleanWordCount,nounCount,verbCount
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...,16,10,1
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni,6,4,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...,23,13,3
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say,9,3,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah dont think goes usf lives around though,8,1,4
5,spam,FreeMsg Hey there darling it's been 3 week's n...,freemsg hey darling 3 weeks word back id like ...,19,9,8
6,ham,Even my brother is not like to speak with me. ...,even brother like speak treat like aids patent,8,3,2
7,ham,As per your request 'Melle Melle (Oru Minnamin...,per request melle melle oru minnaminunginte nu...,16,12,4
8,spam,WINNER!! As a valued network customer you have...,winner valued network customer selected receiv...,18,11,6
9,spam,Had your mobile 11 months or more? U R entitle...,mobile 11 months u r entitled update latest co...,18,11,3


In [79]:
#now we go about doing the normal feature engineering
#lets look at some advanced features
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

def extraFeatures(processedData):
    #lets create their objects
    cvz = CountVectorizer()
    #we will fit the clean data over here
    cvz.fit(processedData["cleanText"].values.astype('U'))
    count_vectors = cvz.transform(processedData["cleanText"].values.astype('U'))
    word_tfidf = TfidfVectorizer(max_features = 500)
    #we will fit the clean data over here
    word_tfidf.fit(processedData["cleanText"].values.astype('U'))
    word_vectors_tfidf = word_tfidf.transform(processedData["cleanText"].values.astype('U'))
    return word_vectors_tfidf


word_vectors_tfid = extraFeatures(processedData)




word_vectors_tfid

<3000x500 sparse matrix of type '<class 'numpy.float64'>'
	with 14076 stored elements in Compressed Sparse Row format>

In [81]:
#we now havea lot of features to play with. We now have to combine it all
# we will create a sparse matrix
from scipy.sparse import hstack, csr_matrix

#we have had a look at the processed data and we will choose the columns accordingly
meta_features = [ 'cleanWordCount',
       'nounCount', 'verbCount']
feature_set1 = processedData[meta_features]
train = hstack([word_vectors_tfid, csr_matrix(feature_set1)], "csr")
train






array([0, 0, 1, ..., 0, 0, 0])

In [85]:
#it is time for classification, but before that we have to label encode the label column
from sklearn.preprocessing import LabelEncoder
target = processedData["label"].values
target = LabelEncoder().fit_transform(target)
from sklearn.model_selection import train_test_split
train_x, validation_x, train_y, validation_y = train_test_split(train, target)


In [86]:
from sklearn import naive_bayes
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn import ensemble
from sklearn.metrics import accuracy_score

In [110]:
f = open('modelDetailsAndSuccess.txt', 'r+')

In [111]:
#we now run different models on it
model = naive_bayes.MultinomialNB()
model.fit(train_x, train_y)
preds = model.predict(validation_x)
print(accuracy_score(preds, validation_y))
print(model)
f.write(str(accuracy_score(preds, validation_y)))
f.write(str(model))
model = LogisticRegression()
model.fit(train_x, train_y)
preds = model.predict(validation_x)
print(accuracy_score(preds, validation_y))
print(model)
f.write(str(accuracy_score(preds, validation_y)))
f.write(str(model))
model = svm.SVC()
model.fit(train_x, train_y)
preds = model.predict(validation_x)
print(accuracy_score(preds, validation_y))
print(model)
f.write(str(accuracy_score(preds, validation_y)))
f.write(str(model))
#ensemble methods
model = ensemble.ExtraTreesClassifier()
model.fit(train_x, train_y)
preds = model.predict(validation_x)
print(accuracy_score(preds, validation_y))
print(model)
f.write(str(accuracy_score(preds, validation_y)))
f.write(str(model))
f.close()

0.9666666666666667
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
0.968
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
0.8986666666666666
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)


/Users/aviralsharma/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/aviralsharma/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/aviralsharma/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.972
ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
                     oob_score=False, random_state=None, verbose=0,
                     warm_start=False)


In [112]:
#It is time to create the final repo and add the results into it
!pachctl create repo RESULTS
!pachctl list repo

NAME        CREATED                SIZE (MASTER) 
RESULTS     Less than a second ago 0B            
CLEANDATA   52 minutes ago         450.9KiB      
SMSTRAINING About an hour ago      268.4KiB      
jup         3 hours ago            0B            
test        24 hours ago           0B            
prep2       27 hours ago           868.1KiB      
prep        28 hours ago           555KiB        
sms         29 hours ago           473.5KiB      
possible    44 hours ago           55B           
inference   44 hours ago           94B           
kim         45 hours ago           1.465KiB      
training    46 hours ago           3.882KiB      


In [113]:
!pachctl put file RESULTS@master:modelDetailsAndSuccess.txt -f modelDetailsAndSuccess.txt --overwrite

In [114]:
'''
4. Now that we have done the SMS spam analysis for for one version of rawData and one version of preprocessed
   data, we finally get to try what we have all been waiting for: APPLYING DATA VERSIONING.
   Just for reference purposes, I will be refering to the 4 forms of raw data as RAW1(3000 rows) RAW2(5572 rows)
   RAW3(5000 rows) and RAW4(4000 rows) and I will be refereing to the more advanced preproccesing as PREPROCESS1
   and the lesser preprocessing method as PREPROCESS2. 
   NOTE: THE PREPROCESSING EXAMPLES USED HERE CAN ALSO BE IMPLIED AS AN ANALOGY FOR MODELS. EVERY MODEL CAN BE
   CODED IN SUCH A WAY THAT CERTAIN VERSIONS ARE MORE ADVANCED(for lack of a better word ~ compact maybe?) THAN 
   OTHERS
   I will be providing the results of all of it the end of it all
'''

'\n4. Now that we have done the SMS spam analysis for for one version of rawData and one version of preprocessed\n   data, we finally get to try what we have all been waiting for: APPLYING DATA VERSIONING.\n   Just for reference purposes, I will be refering to the 4 forms of raw data as RAW1(3000 rows) RAW2(5572 rows)\n   RAW3(5000 rows) and RAW4(4000 rows) and I will be refereing to the more advanced preproccesing as PREPROCESS1\n   and the lesser preprocessing method as PREPROCESS2. \n   NOTE: THE PREPROCESSING EXAMPLES USED HERE CAN ALSO BE IMPLIED AS AN ANALOGY FOR MODELS. EVERY MODEL CAN BE\n'

In [115]:
#RAW1 PREPROCESS 1
#we first need to access the previous version of the CLEANDATA
!pachctl list commit CLEANDATA

REPO      BRANCH COMMIT                           PARENT                           STARTED           DURATION           SIZE     
CLEANDATA master a6f41f8ba4a54824a5a9e4bf948a9bb5 b290f1d7f77c487cb4bcc066ba731ab5 58 minutes ago    Less than a second 450.9KiB 
CLEANDATA master b290f1d7f77c487cb4bcc066ba731ab5 <none>                           About an hour ago Less than a second 483.7KiB 


In [116]:
!pachctl get file CLEANDATA@b290f1d7f77c487cb4bcc066ba731ab5:processedData.csv > raw1pre1.csv

In [118]:
#Now read the data normally through the command .read_csv()
rawData = pd.read_csv("raw1pre1.csv", index_col = False)
rawData = rawData.drop("Unnamed: 0", axis = 1)

In [121]:
def extraFeatures(processedData):
    #lets create their objects
    cvz = CountVectorizer()
    #we will fit the clean data over here
    cvz.fit(processedData["cleanText"].values.astype('U'))
    count_vectors = cvz.transform(processedData["cleanText"].values.astype('U'))
    word_tfidf = TfidfVectorizer(max_features = 500)
    #we will fit the clean data over here
    word_tfidf.fit(processedData["cleanText"].values.astype('U'))
    word_vectors_tfidf = word_tfidf.transform(processedData["cleanText"].values.astype('U'))
    return word_vectors_tfidf


word_vectors_tfid = extraFeatures(rawData)
#we have had a look at the processed data and we will choose the columns accordingly
meta_features = [ 'rawWordCount', 'cleanWordCount',
       'characterCount', 'characterCountWithoutSpace', 'digitOccurrece',
       'nounCount', 'verbCount']
feature_set1 = rawData[meta_features]
train = hstack([word_vectors_tfid, csr_matrix(feature_set1)], "csr")


train_x, validation_x, train_y, validation_y = train_test_split(train, target)

f = open('modelDetailsAndSuccess.txt', 'r+')
#we now run different models on it
model = naive_bayes.MultinomialNB()
model.fit(train_x, train_y)
preds = model.predict(validation_x)
print(accuracy_score(preds, validation_y))
print(model)
f.write(str(accuracy_score(preds, validation_y)))
f.write(str(model))
model = LogisticRegression()
model.fit(train_x, train_y)
preds = model.predict(validation_x)
print(accuracy_score(preds, validation_y))
print(model)
f.write(str(accuracy_score(preds, validation_y)))
f.write(str(model))
model = svm.SVC()
model.fit(train_x, train_y)
preds = model.predict(validation_x)
print(accuracy_score(preds, validation_y))
print(model)
f.write(str(accuracy_score(preds, validation_y)))
f.write(str(model))
#ensemble methods
model = ensemble.ExtraTreesClassifier()
model.fit(train_x, train_y)
preds = model.predict(validation_x)
print(accuracy_score(preds, validation_y))
print(model)
f.write(str(accuracy_score(preds, validation_y)))
f.write(str(model))
f.close()

0.9706666666666667
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
0.964
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
0.9293333333333333
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
0.964
ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fra

/Users/aviralsharma/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/aviralsharma/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/aviralsharma/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [122]:
#forgot to concise it, putting everything in a big function now:
def concise(rawData, meta_features):
    word_vectors_tfid = extraFeatures(rawData)
    #we have had a look at the processed data and we will choose the columns accordingly
    feature_set1 = rawData[meta_features]
    train = hstack([word_vectors_tfid, csr_matrix(feature_set1)], "csr")


    train_x, validation_x, train_y, validation_y = train_test_split(train, target)

    f = open('modelDetailsAndSuccess.txt', 'r+')
    #we now run different models on it
    model = naive_bayes.MultinomialNB()
    model.fit(train_x, train_y)
    preds = model.predict(validation_x)
    print(accuracy_score(preds, validation_y))
    print(model)
    f.write(str(accuracy_score(preds, validation_y)))
    f.write(str(model))
    model = LogisticRegression()
    model.fit(train_x, train_y)
    preds = model.predict(validation_x)
    print(accuracy_score(preds, validation_y))
    print(model)
    f.write(str(accuracy_score(preds, validation_y)))
    f.write(str(model))
    model = svm.SVC()
    model.fit(train_x, train_y)
    preds = model.predict(validation_x)
    print(accuracy_score(preds, validation_y))
    print(model)
    f.write(str(accuracy_score(preds, validation_y)))
    f.write(str(model))
    #ensemble methods
    model = ensemble.ExtraTreesClassifier()
    model.fit(train_x, train_y)
    preds = model.predict(validation_x)
    print(accuracy_score(preds, validation_y))
    print(model)
    f.write(str(accuracy_score(preds, validation_y)))
    f.write(str(model))
    f.close()

In [123]:
#Now let us tackle RAW 2, PREPROCESS 1 & 2
m1  = [ 'rawWordCount', 'cleanWordCount',
       'characterCount', 'characterCountWithoutSpace', 'digitOccurrece',
       'nounCount', 'verbCount']
m2 = [ 'cleanWordCount',
       'nounCount', 'verbCount']

#again lets look at the commits of TRAINING
!pachctl list commit SMSTRAINING

REPO        BRANCH COMMIT                           PARENT                           STARTED     DURATION           SIZE     
SMSTRAINING master 1c81adf333904bb8afc7175474f27a21 7f18d48fa86a401896ed2adb0f1fa118 2 hours ago Less than a second 268.4KiB 
SMSTRAINING master 7f18d48fa86a401896ed2adb0f1fa118 534750817c0846639aeb930432a45d90 2 hours ago Less than a second 494.2KiB 
SMSTRAINING master 534750817c0846639aeb930432a45d90 d5627bffdfb34ea0acbbc53388e4d6b0 2 hours ago Less than a second 444.5KiB 
SMSTRAINING master d5627bffdfb34ea0acbbc53388e4d6b0 <none>                           2 hours ago Less than a second 354.6KiB 


In [125]:
!pachctl get file SMSTRAINING@7f18d48fa86a401896ed2adb0f1fa118:raw.csv >raw2.csv

In [152]:
#Now read the data normally through the command .read_csv()
rawData = pd.read_csv("raw2.csv", index_col = False)
rawData = rawData.drop("Unnamed: 0", axis = 1)


In [153]:
rawDataPre1 = preProcessAndFeature(rawData)
rawDataPre2 = preProcessAndFeatureVersion2(rawData)


In [154]:
concise(rawDataPre1, m1)
concise(rawDataPre1, m1)

0.9741564967695621
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
0.9791816223977028
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
0.9519023689877961
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
0.9827709978463748
ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
         

In [155]:
'''
NOTE: it is pretty obvious that I am not storing the preprocessed data in CLEANDATA repo nor am I storing the other
      results in RESULTS as that would make this notebook even longer. As a result, I am just focusing on the data
      versioning for just one repo. I already showed the data versioning for CLEANDATA for RAW1. It is the same concept
      for all. look at the commits for each repo and focus on the commit ID.
'''

'\nNOTE: it is pretty obvious that I am not storing the preprocessed data in CLEANDATA repo nor am I storing the other\n      results in RESULTS as that would make this notebook even longer. As a result, I am just focusing on the data\n      versioning for just one repo. I already showed the data versioning for CLEANDATA for RAW1. It is the same concept\n      for all. look at the commits for each repo and focus on the commit ID.\n'

In [156]:
#Now let us tackle RAW 3, PREPROCESS 1 & 2
!pachctl list commit SMSTRAINING

REPO        BRANCH COMMIT                           PARENT                           STARTED     DURATION           SIZE     
SMSTRAINING master 1c81adf333904bb8afc7175474f27a21 7f18d48fa86a401896ed2adb0f1fa118 3 hours ago Less than a second 268.4KiB 
SMSTRAINING master 7f18d48fa86a401896ed2adb0f1fa118 534750817c0846639aeb930432a45d90 3 hours ago Less than a second 494.2KiB 
SMSTRAINING master 534750817c0846639aeb930432a45d90 d5627bffdfb34ea0acbbc53388e4d6b0 3 hours ago Less than a second 444.5KiB 
SMSTRAINING master d5627bffdfb34ea0acbbc53388e4d6b0 <none>                           3 hours ago Less than a second 354.6KiB 


In [157]:
!pachctl get file SMSTRAINING@534750817c0846639aeb930432a45d90:raw.csv >raw3.csv

In [158]:
#Now read the data normally through the command .read_csv()
rawData = pd.read_csv("raw3.csv", index_col = False)
rawData = rawData.drop("Unnamed: 0", axis = 1)

In [159]:
rawDataPre1 = preProcessAndFeature(rawData)
rawDataPre2 = preProcessAndFeatureVersion2(rawData)
concise(rawDataPre1, m1)
concise(rawDataPre2, m2)

0.98
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
0.976
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
0.936
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
0.9792
ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimato

In [160]:
#Now let us tackle RAW 4, PREPROCESS 1 & 2
!pachctl get file SMSTRAINING@d5627bffdfb34ea0acbbc53388e4d6b0:raw.csv >raw_4.csv
#Now read the data normally through the command .read_csv()
rawData = pd.read_csv("raw_4.csv", index_col = False)
rawData = rawData.drop("Unnamed: 0", axis = 1)

In [161]:
rawDataPre1 = preProcessAndFeature(rawData)
rawDataPre2 = preProcessAndFeatureVersion2(rawData)
concise(rawDataPre1, m1)
concise(rawDataPre2, m2)

0.979
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
0.973
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
0.927
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
0.979
ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimato

In [162]:
'''
Conclusion (Finally :D ):
The advantage of data versioning might not have been that visible from this experiment but that may be because
1. the data set is too small
2. I played around the data-set and the process related to it (namely:preprocessing and feature engineering)
   The difference might be more subtle if we play around with the model instead which is also possible
   
The results:
RAW1:
PREPROCESS 2
0.9666666666666667
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
0.968LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
0.8986666666666666
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
0.972
ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
                     oob_score=False, random_state=None, verbose=0,
                     warm_start=False)
                     
PREPROCESS 1
0.9706666666666667
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
0.964
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
0.9293333333333333
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
0.964
ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
                     oob_score=False, random_state=None, verbose=0,
                     warm_start=False)
​
RAW2:
​
PREPROCESS1:
0.9720028715003589
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
0.9720028715003589
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
0.9339554917444365
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
0.9813352476669059
ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
                     oob_score=False, random_state=None, verbose=0,
                     warm_start=False)
​
PREPROCESS2:
0.9705671213208902
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
0.9741564967695621
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
0.9023689877961235
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
0.9712849964106246
ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
                     oob_score=False, random_state=None, verbose=0,
                     warm_start=False)
​
RAW3:
​
PREPROCESS1:
0.9648
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
0.9696
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
0.944
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
0.9808
ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
                     oob_score=False, random_state=None, verbose=0,
                     warm_start=False)
​
PREPROCESS2:
0.9648
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
0.9712
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
0.9056
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
0.9768
ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
                     oob_score=False, random_state=None, verbose=0,
                     warm_start=False)
​
​
RAW4:
​
PREPROCESS1:
0.982
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
0.975
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
0.945
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
0.976
ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
                     oob_score=False, random_state=None, verbose=0,
                     warm_start=False)
​
PREPROCESS2:
0.965
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
0.967
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
0.907
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
0.975
ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
                     oob_score=False, random_state=None, verbose=0,
                     warm_start=False)
​
​
​
​
​
RAW 2 (preprocess 1) which was the best data set performed the best for ensembling with an accuracy of 
0.9813352476669059 which is what I accepted from the start.
'''

"\nConclusion (Finally :D ):\nThe advantage of data versioning might not have been that visible from this experiment but that may be because\n1. the data set is too small\n2. I played around the data-set and the process related to it (namely:preprocessing and feature engineering)\n   The difference might be more subtle if we play around with the model instead which is also possible\n   \nThe results:\nRAW1:\nPREPROCESS 2\n0.9666666666666667\nMultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)\n0.968LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,\n                   intercept_scaling=1, l1_ratio=None, max_iter=100,\n                   multi_class='warn', n_jobs=None, penalty='l2',\n                   random_state=None, solver='warn', tol=0.0001, verbose=0,\n                   warm_start=False)\n0.8986666666666666\nSVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,\n    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',\n    kernel